In [ ]:
import os

import pandas as pd
from tqdm import tqdm

In [7]:
keywords_gt = pd.read_csv(os.path.join('..', 'data', 'keywords_gt_with_examples.csv'))

In [8]:
data = keywords_gt[
    (keywords_gt['n_keywords'] == 2) & 
    (keywords_gt['presence'] == 1) &
    (keywords_gt['result_count'] >= 15) & 
    (keywords_gt['r_id_index'] < 20)
].iloc[:1].reset_index(drop=True)

data

,country_tag,type,difficulty,r_rsc_id,s_rsc_id,r_pkg_id,s_pkg_id,r_col_name,s_col_name,nl,top_k,n_keywords,keywords,presence,result_count,r_id_index,s_id_index,prompt_tokens,completion_tokens,response
0,CAN,multi-table-join,challenging,a7c95445-e336-41e3-945c-1bc938ff0612,ae2204c7-db9e-450e-8158-c70eedada7ae,12c89aa5-063a-f0ed-334a-3019133b9ee8,4879e498-c9ac-44a3-d7d1-21af22e2224c,department_/_minista_re,department_/_minista_re,How has the payment amount changed for the sam...,1000,2,suppliers+payments,1,17,14.0,12.0,648,15,<think>\n\n</think>\n\n<keywords>suppliers+pay...


## Fetching Packages and relative Tables

In [1]:
from ckan import CanadaCKAN

ckan_client = CanadaCKAN() 

In [2]:
result = ckan_client.resource_show('e801d617-497e-4a18-b6ce-85e7c81eb471')

In [3]:
result


{'help': 'https://open.canada.ca/data/api/3/action/help_show?name=resource_show',
 'success': True,
 'result': {'cache_last_updated': None,
  'cache_url': None,
  'character_set': 'UTF-8',
  'created': '2023-06-02T12:36:26.740000',
  'data_quality': [],
  'datastore_active': False,
  'datastore_contains_all_records_of_source_file': False,
  'date_published': '2023-06-02',
  'description': None,
  'format': 'CSV',
  'hash': '',
  'id': 'e801d617-497e-4a18-b6ce-85e7c81eb471',
  'language': ['en', 'fr'],
  'last_modified': '2025-06-28T00:00:00',
  'metadata_modified': '2025-06-29T05:10:45.143905',
  'mimetype': None,
  'mimetype_inner': None,
  'name': 'Contract history, 2020-2021',
  'name_translated': {'fr': 'Contrats octroyés, 2020-2021',
   'en': 'Contract history, 2020-2021'},
  'package_id': '4fe645a1-ffcd-40c1-9385-2c771be956a4',
  'position': 15,
  'resource_type': 'dataset',
  'state': 'active',
  'unique_identifier': '',
  'url': 'https://canadabuys.canada.ca/opendata/pub/2020-2

In [2]:
import os
import shutil

shutil.rmtree('tmp')
os.makedirs('tmp', exist_ok=True)

cnt = ckan_client.download_tables_from_package_search('tmp', 'csv', 3, q='suppliers+payments', rows=20)

cnt

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/nanni/miniconda3/envs/lakegen/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/home/nanni/miniconda3/envs/lakegen/lib/python3.12/concurrent/futures/process.py", line 388, in run
    self.join_executor_internals()
  File "/home/nanni/miniconda3/envs/lakegen/lib/python3.12/concurrent/futures/process.py", line 570, in join_executor_internals
    self._join_executor_internals()
  File "/home/nanni/miniconda3/envs/lakegen/lib/python3.12/concurrent/futures/process.py", line 587, in _join_executor_internals
    p.join()
  File "/home/nanni/miniconda3/envs/lakegen/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nanni/miniconda3/envs/lakegen/lib/python3.12/multiprocessing/popen_fork.py", line 43, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
           ^^^^^^^^^^^

'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType' object has no attribute 'headers'
'NoneType'

0

## Setup the BLEND index with ALL the downloaded resources

In [1]:
import blend

indexer = blend.BLEND('blend.db')

In [ ]:
_, dbcon = indexer.create_index('tmp', max_workers=6, limit_rows_per_table=1000, verbose=True)

┌───────────┬─────────┬──────────┬────────┬──────────────────┬─────────────┬──────────┬──────────┐
│ CellValue │ TableId │ ColumnId │ RowId  │ GranularityClass │ Granularity │ Quadrant │ SuperKey │
│  varchar  │ varchar │  uint32  │ uint32 │     varchar      │   varchar   │ boolean  │   blob   │
├───────────┴─────────┴──────────┴────────┴──────────────────┴─────────────┴──────────┴──────────┤
│                                             0 rows                                             │
└────────────────────────────────────────────────────────────────────────────────────────────────┘

None


Parsing and storing tables:   0%|          | 0/45 [00:06<?, ?it/s]


In [5]:
dbcon.table('AllTables').show()

ConnectionException: Connection Error: Connection already closed!

In [3]:
dbcon.sql("SELECT COUNT(*) FROM AllTables;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         2953 │
└──────────────┘

In [ ]:
s_rsc_metadata = ckan_client.resource_show(resource_id=data['s_rsc_id'][0])

In [ ]:
response = ckan_client.package_search("payment+supplier", 3)

## Setup an Agent for Joinable Column Selection

In [ ]:
from dotenv import load_dotenv
from autogen_core.models import UserMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [ ]:
# same seed for any random operation
seed = 42

# Load environment variables from .env file
load_dotenv()
assert "TOGETHER_API_KEY" in os.environ

settings = {
    "model": "Qwen/Qwen3-235B-A22B-fp8-tput",
    "temperature": 0.1,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "max_tokens": 10000,
}

model_client = OpenAIChatCompletionClient(
    base_url = "https://api.together.xyz/v1/",
    api_key=os.getenv("TOGETHER_API_KEY", ""),
    model_info= {
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
        "structured_output": True
    },
    **settings   
)

NameError: name 'load_dotenv' is not defined

In [ ]:
with open(os.path.join('..', 'backend', 'prompts', 'JoinSelector.md')) as file:
    JOIN_SELECTOR_PROMPT_TEMPLATE = file.read()

In [ ]:
analyzer = AssistantAgent(
    name="QueryAnalyzer", model_client=model_client, system_message=formatted_prompt
)

response = await analyzer.run(task=question)
response = response.messages[-1]
